In [1]:
%%configure -f
{
    "conf":{
        "spark.executor.instances": "2",
        "spark.executor.memory": "4g",
        "spark.executor.cores": "2"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
707,application_1765289937462_0700,pyspark,idle,Link,Link,None,
716,application_1765289937462_0709,pyspark,idle,Link,Link,None,
717,application_1765289937462_0710,pyspark,idle,Link,Link,None,
719,application_1765289937462_0712,pyspark,idle,Link,Link,None,
720,application_1765289937462_0713,pyspark,idle,Link,Link,None,
729,application_1765289937462_0722,pyspark,idle,Link,Link,None,
730,application_1765289937462_0723,pyspark,idle,Link,Link,None,
735,application_1765289937462_0728,pyspark,idle,Link,Link,None,
736,application_1765289937462_0729,pyspark,idle,Link,Link,None,
737,application_1765289937462_0730,pyspark,idle,Link,Link,None,


In [2]:
from sedona.spark import *
from pyspark.sql import SparkSession


from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType
from pyspark.sql.functions import udf, year, avg, count, concat, lit, round, rank, col

spark = SparkSession \
    .builder \
    .appName("query 4 execution") \
    .getOrCreate()

# Create sedona context
sedona = SedonaContext.create(spark)

Starting Spark application


The code failed because of a fatal error:
	Session 741 did not start up in 60 seconds..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [3]:
crime_df1 = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2010_2019.csv", header = True, inferSchema = True)
crime_df2 = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2020_2025.csv", header = True, inferSchema = True)
crime_df = crime_df1.union(crime_df2)

The code failed because of a fatal error:
	Session 741 did not start up in 60 seconds..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [4]:
stations_df = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Police_Stations.csv", header = True, inferSchema = True)
stations_df.head()

The code failed because of a fatal error:
	Session 741 did not start up in 60 seconds..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [5]:
from pyspark.sql.window import Window

# Window: nearest station per crime
window = Window.partitionBy("DR_NO").orderBy("distance")

# Filter crimes (drop Null Island etc.)
filtered_crimes_df = (
    crime_df
    .filter(col("LAT").isNotNull() & col("LON").isNotNull())
    .filter(~((col("LAT") == 0) & (col("LON") == 0)))   # Null Island
    .select("DR_NO", "LAT", "LON")
)
#define function for counting distance with sedona
def get_distance(lat1_col, lon1_col, lat2_col, lon2_col):
    # Sedona ST_Point(x, y) = (longitude, latitude)
    p1 = ST_Point(lon1_col.cast("double"), lat1_col.cast("double"))
    p2 = ST_Point(lon2_col.cast("double"), lat2_col.cast("double"))
    return ST_DistanceSphere(p1, p2) / 1000.0  # km

The code failed because of a fatal error:
	Session 741 did not start up in 60 seconds..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [6]:
import time

start = time.time()
# Cross join with stations, compute distance to each station seperately
crimes_stations_dist_df = (
    filtered_crimes_df
    .crossJoin(
        stations_df.select(
            col("division"),  
            col("Y"), # Lat
            col("X") # lon
        )
    )
    .withColumn(
        "distance",
        get_distance(col("LAT"), col("LON"), col("Y"), col("X"))
    )
)

# Keep only nearest station per crime
nearest_station_df = (
    crimes_stations_dist_df
    .withColumn("distance_rank", rank().over(window))
    .filter(col("distance_rank") == 1) #nearest
    .select("division", "distance")
)

# Aggregate per division: count + average distance, sort by count desc
division_stats_df = (
    nearest_station_df
    .groupBy("division")
    .agg(
        avg("distance").alias("average_distance_km"), 
        count("*").alias("#") # only for output consistency
    )
    .withColumn(
        "average_distance_km",
        round(col("average_distance_km"), 3) #round to 3 decimal
    )
    .orderBy(col("#").desc())
)

division_stats_df.show(100, truncate=False)

end = time.time()
print("Execution time:", end - start, "seconds")


The code failed because of a fatal error:
	Session 741 did not start up in 60 seconds..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.
